# Cyber security detection

In this practical session, we will embark on an insightful journey into the world of Machine Learning (ML) with a focus on cybersecurity, specifically within Supervisory Control and Data Acquisition (SCADA) networks. Our goal is to develop a machine learning model capable of detecting cybersecurity issues in SCADA networks. This exercise will not only enhance your understanding of ML in cybersecurity but also provide you with hands-on experience in safeguarding critical infrastructure.

## Understanding SCADA Systems

SCADA systems are vital for the operation and monitoring of industrial processes. These systems are used to control and supervise equipment and processes in various sectors, including energy, water treatment, and transportation. For instance, in the energy sector, SCADA systems manage the distribution of electricity in power grids, ensuring that the supply meets demand. In water treatment plants, they monitor and control the flow and treatment of water, ensuring safety and efficiency.

SCADA networks are critical because they support the seamless operation of our infrastructure, making them prime targets for cyber-attacks. The consequences of a successful attack can range from operational disruptions to significant financial losses and potential threats to public safety.

## Dataset Overview

The dataset you will be working with has been meticulously compiled from a SCADA system testbed, designed to emulate real-world industrial systems. This testbed was subjected to various cyber-attacks, particularly reconnaissance attacks, where attackers scan the network to identify vulnerabilities that could be exploited in future attacks.

Here are the key columns in the dataset:

**Source Port (Sport)**: This represents the port number of the source in the network transaction, crucial for understanding the communication patterns and potential unauthorized access points.

**Total Packets (TotPkts)**: This column shows the total number of packets exchanged in a transaction, which can indicate the volume and intensity of network communication.

**Total Bytes (TotBytes)**: Reflects the total amount of data transferred during the transaction, important for detecting large or unusual data transfers.

**Source Packets (SrcPkts) and Destination Packets (DstPkts)**: These columns provide counts of packets sent from the source to the destination and vice versa, useful for identifying asymmetric traffic flows often seen in cyber-attacks.

**Source Bytes (SrcBytes)**: This column indicates the amount of data sent from the source, helping to identify potential data exfiltration attempts.

**Target** -> Label: Each record is labeled as either part of a network attack or normal activity. This labeling is crucial for training our ML model to differentiate between benign and malicious network behaviors.


Source of the dataset: https://www.cse.wustl.edu/~jain/iiot/index.html

You can download the dataset here: https://drive.google.com/file/d/12JNZjMbYucd-TfcFoz9AfNhv6dg7WAzk/view?usp=sharing


In [4]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

1) Read the data with pandas (the dataset is about 200Mb)

Number of samples: 7_037_983

In [5]:
df = pd.read_csv("wustl-scada-2018.csv")
df

,Sport,TotPkts,TotBytes,SrcPkts,DstPkts,SrcBytes,Target
0,143,2,180,2,0,180,0
1,68,2,684,2,0,684,0
2,0,1,60,1,0,60,0
3,54949,10,628,4,6,248,0
4,54943,8,496,4,4,248,0
...,...,...,...,...,...,...,...
7037978,49317,14,904,8,6,520,0
7037979,49318,14,904,8,6,520,0
7037980,49319,12,780,8,4,520,0
7037981,49320,12,780,8,4,520,0


2. Identify categorical and numerical features for your ML model

In [6]:
# Categorical
# ----------
# Sport
categorical = ['Sport']

# Numerical,
# ---------
# TotPkts
# TotBytes
# SrcPkts
# DstPkts
# SrcBytes
numerical = ['TotPkts', 'TotBytes', 'SrcPkts', 'DstPkts', 'SrcBytes']

3. Compute some statistics on the dataset,

- Label proportion
- Proportion of NULL values
- Count the number of modalities for each categorical features
- Check label proportion for some specific modality of a categorical feature
- Chose a column and compute some statistics like mean, std ...
- Display the distribution of a continuous feature (bucketization of the feature and count)
- Display the label rate per bucket for all the numerical features (what can we understand from those plots)

In [7]:
# Label proportion
100 * df['Target'].sum() / len(df)

5.731784234204601

In [9]:
df.columns

Index(['Sport', 'TotPkts', 'TotBytes', 'SrcPkts', 'DstPkts', 'SrcBytes',
       'Target'],
      dtype='object')

In [19]:
# Proportion of NULL values
for col in df.columns:
    print(f"{col} -> {df[col].isna().sum()}")

Sport -> 0
TotPkts -> 0
TotBytes -> 0
SrcPkts -> 0
DstPkts -> 0
SrcBytes -> 0
Target -> 0


In [20]:
# Count the number of modalities for each categorical features
df['Sport'].unique(), len(df['Sport'].unique())

(array([   143,     68,      0, ...,  33354,  65535, 132186]), 24608)

In [25]:
# Check label proportion for some specific modality of a categorical feature
df_sport143 = df[df['Sport'] == 68]
100 * df_sport143['Target'].sum() / len(df_sport143)

0.0

In [26]:
# Chose a column and compute some statistics like mean, std ...
df['SrcBytes'].mean(), df['SrcBytes'].std(), df['SrcBytes'].min(), df['SrcBytes'].max()

(1461.5397144323879, 69873.73153999464, 0, 16153618)

In [27]:
df['SrcBytes'].describe()

count    7.037983e+06
mean     1.461540e+03
std      6.987373e+04
min      0.000000e+00
25%      6.440000e+02
50%      6.440000e+02
75%      6.440000e+02
max      1.615362e+07
Name: SrcBytes, dtype: float64

In [30]:
df['SrcBytes'].min(), df['SrcBytes'].max()

(0, 16153618)

In [31]:
df.columns

Index(['Sport', 'TotPkts', 'TotBytes', 'SrcPkts', 'DstPkts', 'SrcBytes',
       'Target', 'binned'],
      dtype='object')

In [34]:
col_name = 'TotBytes'

In [45]:
df['TotBytes'].min(), df['TotBytes'].max(), df['TotBytes'].mean(), df['TotBytes'].std()

(60, 2144931776, 104499.6265270888, 11864894.643184064)

In [46]:
(df['TotBytes'].max() - df['TotBytes'].min()) / 100

21449317.16

In [82]:
bins = [0, 100, 150, 200, 250, 300, 600, 900, 1100, 1140, 1150, 1151, 1152, 1153, 1154, 1155, 1156, 1157, 1158, 1160, 1170, 1180, 1190, 1200, 1210, 1220, 1230, 1240, 1250, 1260, 1300, 1400, 1500, 1600]

In [83]:
# Display the distribution of a continuous feature (bucketization of the feature and count)
num_quantiles = 5

df['binned'] = pd.cut(df[col_name], bins=bins, include_lowest=True, right=True)
bin_labels = [f'{bins[i]} to {bins[i+1]}' for i in range(len(bins)-1)]
bin_counts = df['binned'].value_counts(sort=False).reindex(pd.IntervalIndex.from_breaks(bins, closed='right')).fillna(0)


fig = px.bar(x=bin_counts.index.astype(str), y=bin_counts.values, labels={'x': 'Buckets', 'y': 'Count'})
fig.update_layout(title='Distribution of Feature by Bucket')
fig.show()

In [84]:
# Display the label rate per bucket for all the numerical features (what can we understand from those plots)

from typing import List

def display_labelrate(feature: str, bins: List):
    df['binned'] = pd.cut(df[feature], bins=bins)

    label_rates = df.groupby('binned')['Target'].mean()

    fig = px.bar(x=label_rates.index.astype(str), y=label_rates.values, labels={'x': 'Buckets', 'y': 'Label Rate'})
    fig.update_layout(title=f'Label Rate per Bucket {feature}')
    fig.show()

In [85]:
display_labelrate('TotPkts', [i for i in range(0, 200, 2)])

In [86]:
display_labelrate('TotBytes', [i for i in range(0, 400, 5)])

In [87]:
display_labelrate('SrcPkts', [i for i in range(0, 100, 1)])

In [88]:
display_labelrate('DstPkts', [i for i in range(0, 100, 1)])

In [89]:
display_labelrate('SrcBytes', [i for i in range(0, 400, 5)])

What can we understand from the data ?

In [ ]:
# What can be understand from the data
# Some buckets are highly correlated with the label

### Split the dataset

Split the dataset into training and test

You can drop the categorical features for now if you don't know how to process categorical features with a huge number of categories

After the split, ensure that we have enough labels

In [8]:
from sklearn.model_selection import train_test_split

In [ ]:
df = df.drop(['binned'], axis=1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(df[numerical], df['Target'], test_size=0.2, random_state=42)

In [14]:
X_train.shape, X_test.shape

((5630386, 5), (1407597, 5))

In [15]:
y_train.shape, y_test.shape

((5630386,), (1407597,))

In [16]:
y_train.sum(), y_test.sum()

(322622, 80780)

### Its time to fit our models

Build 3 different models:
- Logistic regression
- Decision tree
- Gaussian Naives bayes

We are going to compare the performances of our 3 models

In [18]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [19]:
# svc_model_linear = SVC(kernel='linear', verbose=True)
# svc_model = SVC(kernel='rbf', verbose=True)
naive_bayes_model = GaussianNB()
decision_tree_model = DecisionTreeClassifier()
log_reg_model = LogisticRegression(verbose=True)

In [20]:
# Fit all the models
naive_bayes_model.fit(X_train, y_train)

GaussianNB()

In [21]:
decision_tree_model.fit(X_train, y_train)

DecisionTreeClassifier()

In [22]:
log_reg_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(verbose=True)

### Compute metrics from scratch

Accuracy / confusion matrix / precision / recall / f1 score

Check if your implementation matches sklearn implementation results


In [29]:
def compute_confusion_matrix_elements(y_true, y_pred):
    TP = sum((y_true == 1) & (y_pred == 1))
    TN = sum((y_true == 0) & (y_pred == 0))
    FP = sum((y_true == 0) & (y_pred == 1))
    FN = sum((y_true == 1) & (y_pred == 0))
    return TP, TN, FP, FN

In [30]:
def compute_metrics(y_true, y_pred):
    TP, TN, FP, FN = compute_confusion_matrix_elements(y_true, y_pred)

    # Accuracy
    accuracy = (TP + TN) / (TP + TN + FP + FN)

    # Precision
    precision = TP / (TP + FP) if (TP + FP) else 0

    # Recall
    recall = TP / (TP + FN) if (TP + FN) else 0

    # F1 Score
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0

    return accuracy, precision, recall, f1_score

In [31]:
from typing import TypeAlias

ModelType: TypeAlias = GaussianNB | DecisionTreeClassifier | LogisticRegression

In [38]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [39]:
def display_metrics_model(
        model: ModelType,
        x_data: np.array,
        y_data: np.array
    ):
    y_pred = model.predict(x_data)
    acc = accuracy_score(y_data, y_pred)
    prec = precision_score(y_data, y_pred)
    rec = recall_score(y_data, y_pred)
    f1 = f1_score(y_data, y_pred)

    print(f"Accuracy : {acc}")
    print(f"Precision: {prec}")
    print(f"Recall   : {rec}")
    print(f"F1       : {f1}")

In [40]:
print("Naive bayes")
print("TRAIN")
display_metrics_model(naive_bayes_model, X_train.to_numpy(), y_train.to_numpy())

Naive bayes
TRAIN


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


Accuracy : 0.06064149065445957
Precision: 0.057492257518524666
Recall   : 1.0
F1       : 0.10873319801590613


In [41]:
print("Naive bayes")
print("TEST")
display_metrics_model(naive_bayes_model, X_test.to_numpy(), y_test.to_numpy())

Naive bayes
TEST


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


Accuracy : 0.06068214126628573
Precision: 0.057578222060342375
Recall   : 1.0
F1       : 0.10888692837900954


In [42]:
print("Decision tree")
print("TRAIN")
display_metrics_model(decision_tree_model, X_train.to_numpy(), y_train.to_numpy())

Decision tree
TRAIN


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Accuracy : 0.9983430976135561
Precision: 0.9719021410044375
Recall   : 0.9999938007947381
F1       : 0.9857478737917623


In [43]:
print("Decision tree")
print("TEST")
display_metrics_model(decision_tree_model, X_test.to_numpy(), y_test.to_numpy())

Decision tree
TEST


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Accuracy : 0.9983724034649122
Precision: 0.97242118188056
Recall   : 1.0
F1       : 0.9860177844505068


In [44]:
print("Logistic regression")
print("TRAIN")
display_metrics_model(log_reg_model, X_train.to_numpy(), y_train.to_numpy())

Logistic regression
TRAIN


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


Accuracy : 0.9960455641940001
Precision: 0.9396493426272077
Recall   : 0.9948856556589445
F1       : 0.9664789245553733


In [45]:
print("Logistic regression")
print("TEST")
display_metrics_model(log_reg_model, X_test.to_numpy(), y_test.to_numpy())

Logistic regression
TEST


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


Accuracy : 0.9959924609103316
Precision: 0.9389011553873293
Recall   : 0.9949121069571676
F1       : 0.9660954807998605


Now compute the roc auc

In [46]:
from sklearn.metrics import roc_auc_score

In [49]:
y_pred_proba = decision_tree_model.predict_proba(X_test)[:, 1]

In [50]:
roc_auc_score(y_test, y_pred_proba)

0.9995566606775542

### Find the optimal threshold to maximize the f1 score

Choose the best model and compute the optimal threshold to maximize f1 score

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

def find_best_threshold_for_f1(y_test, y_pred_proba):
    best_score = 0
    best_threshold = 0.5

    for threshold in np.arange(0, 1.01, 0.01):
        y_pred = (y_pred_proba >= threshold).astype(int)

        score = f1_score(y_test, y_pred)

        if score > best_score:
            best_score = score
            best_threshold = threshold

    return best_threshold, best_score